# Vietnamese Fact-Checking System - Colab Setup

This notebook sets up the environment for running the fact-checking system on Google Colab Pro.

## 1. Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. Clone Repository or Upload Files

In [ ]:
import os

# Option 1: Clone from repository
# !git clone https://github.com/your-repo/vietnamese-fact-checking.git
# %cd vietnamese-fact-checking

# Option 2: Copy from Google Drive
project_path = '/content/drive/MyDrive/vietnamese-fact-checking'
if not os.path.exists(project_path):
    !mkdir -p {project_path}
    print(f"Created project directory: {project_path}")
else:
    print(f"Project directory exists: {project_path}")

%cd {project_path}

## 3. Install Dependencies

In [ ]:
!pip install -q -r requirements.txt

## 4. Setup Environment Variables

In [ ]:
import os
from google.colab import userdata

# Load API keys from Colab secrets
# Go to the key icon on the left sidebar to add secrets
try:
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_SEARCH_ENGINE_ID'] = userdata.get('GOOGLE_SEARCH_ENGINE_ID')
    os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
    os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
    print("API keys loaded successfully")
except Exception as e:
    print(f"Warning: Could not load some API keys: {e}")
    print("Please add your API keys to Colab secrets")

## 5. Initialize System

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/vietnamese-fact-checking')

from src.config import load_config
from src.logging_config import setup_logging
from src.checkpoint_manager import CheckpointManager, mount_google_drive

# Setup logging
logger = setup_logging(
    log_level="INFO",
    log_dir="/content/drive/MyDrive/vietnamese-fact-checking/logs",
    log_to_file=True,
    log_to_console=True
)

# Load configuration
config = load_config("config.yaml")
print("Configuration loaded successfully")

# Initialize checkpoint manager
checkpoint_manager = CheckpointManager(
    checkpoint_dir=config.colab.drive_mount_path + "/checkpoints",
    interval_minutes=config.colab.checkpoint_interval_minutes,
    auto_start=True
)
print(f"Checkpoint manager initialized (auto-save every {config.colab.checkpoint_interval_minutes} minutes)")

## 6. Check GPU Availability

In [ ]:
import torch

if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("No GPU available. Please enable GPU in Runtime > Change runtime type")

## 7. Test System Components

In [ ]:
# Test checkpoint manager
checkpoint_manager.update_state('test_key', 'test_value')
checkpoint_path = checkpoint_manager.save_checkpoint('test_checkpoint')
print(f"Test checkpoint saved: {checkpoint_path}")

# Test loading
loaded_data = checkpoint_manager.load_checkpoint(checkpoint_path)
print(f"Test checkpoint loaded: {loaded_data['state']}")

print("\nSystem setup complete! Ready to run fact-checking tasks.")